In [2]:
install.packages('datarium')

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


In [3]:
options(repr.plot.width = 8, repr.plot.height = 8)
# Load the data
data('marketing', package = 'datarium')
# Inspect the data and summary
head(marketing); dim(marketing); summary(marketing)

youtube,facebook,newspaper,sales
<dbl>,<dbl>,<dbl>,<dbl>
276.12,45.36,83.04,26.52
53.40,47.16,54.12,12.48
20.64,55.08,83.16,11.16
181.80,49.56,70.20,22.20
216.96,12.96,70.08,15.48
10.44,58.68,90.00,8.64


[1] 200   4

    youtube          facebook       newspaper          sales      
 Min.   :  0.84   Min.   : 0.00   Min.   :  0.36   Min.   : 1.92  
 1st Qu.: 89.25   1st Qu.:11.97   1st Qu.: 15.30   1st Qu.:12.45  
 Median :179.70   Median :27.48   Median : 30.90   Median :15.48  
 Mean   :176.45   Mean   :27.92   Mean   : 36.66   Mean   :16.83  
 3rd Qu.:262.59   3rd Qu.:43.83   3rd Qu.: 54.12   3rd Qu.:20.88  
 Max.   :355.68   Max.   :59.52   Max.   :136.80   Max.   :32.40  

In [4]:
# Visualize the relationship between sales unit and advertising
# budget across different advertising media
library(dplyr)
library(ggplot2)

# Plots scatterplot matrix, variables along rows are on the y-axis, 
# variables along columns are on the x-axis 
pairs(marketing, main='Matrix scatterplot of marketing dataset', pch=21)



Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [5]:
# Perform kNN regression
library(caret)

# Splitting the data set using 80/20 rule
set.seed(100)
trg_idx <- sample(nrow(marketing), nrow(marketing)*0.8)
trg_data <- marketing[trg_idx, ]; head(trg_data)
test_data <- marketing[-trg_idx, ]; head(test_data)

# Fit model on training set
set.seed(101)
mkt_model_knn <- train(
sales~., data = trg_data, method = "knn",  
trControl = trainControl("cv", number = 3),  
preProcess = c("center","scale"),  # Standardize observations as distance measure in kNN is sensitive to scale
tuneLength = 10
)
plot(mkt_model_knn)

# Best tuning parameter k that minimize the RMSE
mkt_model_knn$bestTune  # k = 5

Loading required package: lattice


,youtube,facebook,newspaper,sales
,<dbl>,<dbl>,<dbl>,<dbl>
62,313.56,51.24,65.64,29.04
52,120.48,11.52,4.32,12.84
110,306.48,32.28,6.60,23.76
12,257.64,28.80,4.80,20.88
92,34.32,1.80,39.60,8.76
95,128.88,16.80,13.08,13.80


,youtube,facebook,newspaper,sales
,<dbl>,<dbl>,<dbl>,<dbl>
10,239.76,3.12,25.44,12.72
14,117.00,9.12,8.64,11.64
15,244.92,39.48,55.20,22.80
16,234.48,57.24,63.48,26.88
19,83.04,24.60,21.96,13.56
20,176.76,28.68,22.92,17.52


k
<int>
5


In [6]:
# Make predictions on the test data using predict()
knn_predictions <- predict(mkt_model_knn, test_data); head(knn_predictions)

# Compute the prediction error RMSE
RMSE(knn_predictions, test_data$sales)

# To visualize the performance of kNN regression, plot predicted sales vs sales in test_data
plot(test_data$sales, knn_predictions, main="Prediction performance of kNN regression")
abline(0,1, col='red')  # Verify how well predicted sales match observed sales on test data

[1] 13.632 11.784 22.296 26.352 12.720 18.312

[1] 1.857428

In [7]:
# Perform linear regression

mkt_lmodel <- lm(sales~., data=trg_data)
summary(mkt_lmodel)

# Make predictions on the test data
lm_predictions <- predict(mkt_lmodel, test_data); head(lm_predictions)

# Compute the prediction error RMSE
RMSE(lm_predictions, test_data$sales)

# Visualize the performance of linear regression
plot(test_data$sales, lm_predictions, main="Prediction performance of linear regression")
abline(0,1, col='red')


Call:
lm(formula = sales ~ ., data = trg_data)

Residuals:
   Min     1Q Median     3Q    Max 
-6.317 -1.048  0.274  1.335  3.304 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  3.664231   0.386345   9.484   <2e-16 ***
youtube      0.045421   0.001425  31.865   <2e-16 ***
facebook     0.188477   0.008959  21.037   <2e-16 ***
newspaper   -0.002680   0.005920  -0.453    0.651    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.902 on 156 degrees of freedom
Multiple R-squared:  0.9129,	Adjusted R-squared:  0.9113 
F-statistic: 545.2 on 3 and 156 DF,  p-value: < 2.2e-16


10       14       15       16       19       20 
15.07427 10.67426 22.08192 24.93289 12.01369 17.03697

[1] 2.441419

In [8]:
# Check residual plots
par(mfrow=c(2,2))
residuals(mkt_lmodel)
plot(mkt_lmodel)

62          52         110          12          92          95 
 1.65185708  1.54375246  0.10874844  0.09819287  3.30377777  1.15053106 
        158          72         105          33         119         167 
 0.07498650  2.09875453  0.45192069  2.31506337  0.47336972 -3.47450268 
         53          75         142         124          38          66 
 2.35676170 -0.41739015  1.05468784  0.08046683 -1.12174222  1.63438965 
        183         125          97         128          96         133 
 2.51889346  0.40002900 -1.16711903  2.55402505  0.73810805 -3.42721522 
         74          30         135         153         195          48 
 1.29424699  1.60013454 -1.23478289  0.26135602  0.90387103  1.77326766 
         84         157          59         160         116         147 
-1.02259247 -0.09839140  2.30911881  0.58710575 -0.38414970 -2.53404786 
        175         103         161          21         188         138 
-2.71308093 -3.39215518  0.21859767 -0.06145609  0.24717229  0.03325034 
        123         130         172          77         121         136 
-2.44607754  2.15177975  0.19506083  2.82156241  1.32130819 -2.97961365 
         32          47          50         164          35          41 
 0.65084196  2.04233425  1.80148308  0.72479224  2.22676219  0.27640676 
         87          37          18         191          85         149 
 0.40871892  2.37801346  1.50097168 -2.13423128  1.12215147 -1.73191903 
         65          90         177          93          61          49 
 1.20286962 -0.25467412  0.27151002  0.36290538  2.75620531 -1.70090541 
        200          60         143          91          54          43 
-2.15202932  0.28930902  1.05028936  1.37736179  1.56268749 -1.08619234 
        176         194         170          79         107         178 
 2.71782053  1.27660756 -3.53694997 -4.33088983  1.22074607 -0.55199211 
        102         197          56         193         109         115 
 0.85479206  1.75916745  2.95467871  1.65258815  1.97360377 -0.88047906 
          5          67         126          29          34          82 
-0.29366046  0.46208129  1.71735528 -0.60079631 -1.78333778 -2.78325941 
        100          23          39         171         180         165 
 0.32285558 -1.10034226  0.18065585  1.12607055  0.22455537  0.92037111 
        169          13           4          78         132         108 
-0.03712646 -1.64819424  1.12540610  0.40762149 -3.39667210  1.85523953 
        166          36          55         182         196         156 
-2.66202479 -5.04890972 -0.20542437 -2.06686955  2.58120403 -2.65297415 
          2         174          76         190           8         162 
-2.35326765 -0.36759502 -3.74162032  0.69508064  1.22854750 -0.31377806 
        114         122         155         173          17          69 
-0.63329760 -1.03480021  0.07797387 -0.10393526 -0.27100981 -0.10817200 
        152          80         145          45          27           3 
 1.91738058  1.54590433  1.55009460 -0.50570598 -0.03938162 -3.60018849 
        179          70          40          88         111         159 
-5.02984453  1.43750444  1.28474983  0.52265931 -1.56446268 -3.74235651 
         51         112          99          94          31         187 
-1.46426913  0.80187597  1.62311892  1.27757994 -0.21060913  0.70284988 
        113         137         192           6           9         199 
 0.20383999 -2.45038941  1.67724653 -6.31708066  1.15527657  2.19168990 
          7          81          57          63         140          11 
 0.01882347  0.36446577 -3.68443474 -1.28524486  1.17422295  1.81898648 
         26          46         120          71          28         101 
-4.32258395 -0.31570270 -0.34868328  0.64732503 -1.37440556 -2.55861734 
        184         117           1         134 
 2.60550430  0.23668099  1.98729808  0.44374491

In [11]:
# Improve fit of linear model

# Visualize the correlation between the outcome sales and each predictor
install.packages("corrplot")
library(corrplot)
corrplot(cor(trg_data), type="upper", method="color",addCoef.col = "black",number.cex = 0.6)

# Remove outliers
marketing_0 <- marketing[-131,]  # Remove outlier #131 from residual plot

# Splitting the data set using 80/20 rule
set.seed(100)
trg_idx_0 <- sample(nrow(marketing_0), nrow(marketing_0)*0.8)
trg_data_0 <- marketing_0[trg_idx_0, ]; head(trg_data_0)
test_data_0 <- marketing_0[-trg_idx_0, ]; head(test_data_0)

# Construct improved linear model
mkt_lmodel_0 <- lm(sales~youtube+facebook+newspaper+I(youtube^2)+I(youtube*newspaper)+I(newspaper^2)+I(youtube*facebook)+I(facebook^2), data=trg_data_0)
summary(mkt_lmodel_0)

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


,youtube,facebook,newspaper,sales
,<dbl>,<dbl>,<dbl>,<dbl>
62,313.56,51.24,65.64,29.04
52,120.48,11.52,4.32,12.84
109,15.72,0.48,30.72,6.36
12,257.64,28.80,4.80,20.88
92,34.32,1.80,39.60,8.76
94,301.08,43.80,86.76,26.64


,youtube,facebook,newspaper,sales
,<dbl>,<dbl>,<dbl>,<dbl>
14,117.00,9.12,8.64,11.64
15,244.92,39.48,55.20,22.80
16,234.48,57.24,63.48,26.88
19,83.04,24.60,21.96,13.56
20,176.76,28.68,22.92,17.52
22,284.88,6.12,28.20,15.00



Call:
lm(formula = sales ~ youtube + facebook + newspaper + I(youtube^2) + 
    I(youtube * newspaper) + I(newspaper^2) + I(youtube * facebook) + 
    I(facebook^2), data = trg_data_0)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.1636 -0.3463  0.0082  0.4397  1.1255 

Coefficients:
                         Estimate Std. Error t value Pr(>|t|)    
(Intercept)             6.174e+00  2.449e-01  25.206  < 2e-16 ***
youtube                 4.965e-02  2.015e-03  24.641  < 2e-16 ***
facebook                3.450e-02  1.128e-02   3.057  0.00265 ** 
newspaper               3.401e-03  6.317e-03   0.538  0.59105    
I(youtube^2)           -8.597e-05  5.368e-06 -16.016  < 2e-16 ***
I(youtube * newspaper) -1.069e-05  1.731e-05  -0.618  0.53759    
I(newspaper^2)         -3.850e-06  5.035e-05  -0.076  0.93915    
I(youtube * facebook)   8.688e-04  2.799e-05  31.039  < 2e-16 ***
I(facebook^2)           1.286e-04  1.758e-04   0.732  0.46559    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘

In [12]:
# Make predictions on the test data
lm_predictions_0 <- predict(mkt_lmodel_0, test_data_0); head(lm_predictions_0)

# Compute the prediction error RMSE
RMSE(lm_predictions_0, test_data_0$sales)

plot(mkt_lmodel_0)

14       15       16       19       20       22 
12.07640 23.17052 27.18554 12.45831 17.79529 15.07768

[1] 0.6283433